# Model training - MobileNetV2

This notebook implements an artificial neural network for facemask detection. The MobileNetV2 architecture was used as a basis for our model. For faster training and ability to perform well without a huge dataset is used transfer learning method. The MobileNetV2 basis will be pretrained.

The libraries for this project:

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt

## Data Preprocessing
ImageDataGenerator provides convenient way to do it.
MobileNetV2 architecture is expecting 224x224 input shape. Normalizing the pixel values. Also some data augmentation for more efficient trainning. ImageDataGenerator needs directory with two folders, one with positive classification (people with masks images) and one with negative classification.

In [ ]:
training_dir = 'dataset' # path to directory with training examples
testing_dir = 'test_data' # path to directory with testing examples
val_dir = 'improving/val_data' # path to directory with validation examples

train_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=40,
    width_shift_range=.2, 
    height_shift_range=.2, 
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    training_dir,
    batch_size=64,
    class_mode='categorical',
    target_size=(224, 224)
)

test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
   testing_dir,
   target_size=(224, 224),
   class_mode='categorical',
   batch_size=64
)

validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
   val_dir,
   target_size=(224, 224),
   class_mode='categorical',
   batch_size=64
)



## Model creation 
Pretrained MobileNetV2 on ImageNet dataset without the top will be a foundation. We will create our own top classifier.

In [ ]:
mobilenet = tf.keras.applications.MobileNetV2(
    input_shape = [224, 224, 3], include_top=False, weights='imagenet')

#freezing the MobileNetV2 layers
for layer in mobilenet.layers:
    layer.trainable = False

#creating top
top = mobilenet.output
top = tf.keras.layers.AveragePooling2D((7,7))(top)
top = tf.keras.layers.Flatten()(top)
top = tf.keras.layers.Dense(200,activation = 'relu')(top)
top = tf.keras.layers.Dropout(0.5)(top)
top = tf.keras.layers.Dense(2,activation = 'softmax')(top)
model = tf.keras.models.Model(inputs = mobilenet.inputs, outputs = top)

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fitting the model

In [ ]:
history = model.fit(train_generator, epochs=10, validation_data = validation_generator, verbose = 1,callbacks=[])

Graphs for further analysis 

In [ ]:
from matplotlib import pyplot as plt

plt.plot(range(1,11), history.history['loss'],'r',label='loss function')
plt.xlabel('number of epochs')
plt.ylabel('loss function')
plt.show()

In [ ]:
plt.plot(range(1,11),history.history['accuracy'],'r',label='accuracy')
plt.plot(range(1,11),history.history['val_accuracy'],'blue',label='val_accuracy')
plt.xlabel('number of epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

## Evaluation on test data

In [ ]:
model.evaluate(test_generator)

Saving the trained model

In [ ]:
model.save('name_of_the_model')